<a href="https://colab.research.google.com/github/maurya-bitlegacy/attacks/blob/main/Improved_genetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U gensim==4.0.0
!pip install python-Levenshtein
!pip install textattack

     |████████████████████████████████| 23.9MB 68.0MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 51kB 6.8MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149820 sha256=80981a871c2a30513bc469e37f5f9184aa7d6c6abf5da9f72e716d0032e22c1f
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein
     |████████████████████████████████| 358kB 17.9MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 20.1MB 161kB/s 
     |████████████████████████████████| 2.1MB 48.8MB/s 
     |████████████████████████████████| 102kB 12.1MB/s 
     |████████████████████████████████| 337kB 49.2MB/s 
     |████████████████████████████████| 225kB 53.3MB/s 
     |████████████████████████████████| 778kB 49

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from gensim.test.utils import get_tmpfile
from gensim.models.fasttext import FastText
model1 = FastText.load("./drive/MyDrive/models/fasttext.model") # Loading our FastText model trained on text8 corpus

In [ ]:
model1.wv.most_similar('horse')

[('horsehair', 0.8985454440116882),
 ('horseshoe', 0.8598332405090332),
 ('horses', 0.854678213596344),
 ('horseshoes', 0.850151002407074),
 ('horseback', 0.8197551965713501),
 ('horseman', 0.8112313747406006),
 ('horseradish', 0.8012595772743225),
 ('horsecollar', 0.782683253288269),
 ('racehorse', 0.777208685874939),
 ('horsemen', 0.7719056606292725)]

# Attacking AlBERT-base-v2-Yelp-Polarity

In [ ]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/albert-base-v2-yelp-polarity")
tokenizer = AutoTokenizer("textattack/albert-base-v2-yelp-polarity")


model_wrapper = HuggingFaceModelWrapper(model, tokenizer) # Defines our target model


dataset = HuggingFaceDataset("yelp_polarity", None, "test") # Change the dataset

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:16<00:00, 28.5MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpt45jz1gm.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


## Attacking using our attack recipe:

### Defining our transformation:

In [ ]:
from textattack.transformations import WordSwap

class Swapper(WordSwap): # For fasttext
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        for i in range(10):
            if word.lower() in model1.wv.most_similar(word)[i][0].lower():
                continue # Don't return a word containing the exact word
            elif word.isupper():
                return [model1.wv.most_similar(word)[i][0].upper()] # Preserving case
            elif word[0].isupper():
                return [model1.wv.most_similar(word)[i][0].capitalize()] # Preserving Capitalization in words
            else:
                return [model1.wv.most_similar(word)[i][0]]
            
        return [model1.wv.most_similar(word)[0][0]]

## Defining our attack recipe:

In [ ]:
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.overlap import MaxWordsPerturbed
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()
# We'll constrain modification of already modified indices and stopwords
constraints = [RepeatModification(), StopwordModification()]
# # We'll use the Greedy search method
search_method = GreedyWordSwapWIR("weighted-saliency")
# Create the goal function using the model

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)


textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


### Attacking and displaying results:

In [ ]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack.attack_dataset(dataset, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:03<01:27,  3.64s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [00:06<01:10,  3.04s/it]

[Succeeded / Failed / Total] 3 / 0 / 3:  12%|█▏        | 3/25 [00:06<00:48,  2.18s/it]

[Succeeded / Failed / Total] 4 / 0 / 4:  16%|█▌        | 4/25 [00:07<00:41,  1.96s/it]

...

[Succeeded / Failed / Total] 18 / 5 / 23:  92%|█████████▏| 23/25 [01:47<00:09,  4.67s/it]

[Succeeded / Failed / Total] 19 / 5 / 24:  96%|█████████▌| 24/25 [01:53<00:04,  4.72s/it]

[Succeeded / Failed / Total] 19 / 6 / 25: 100%|██████████| 25/25 [01:54<00:00,  4.59s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 19     |
| Number of failed attacks:     | 6      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 24.0%  |
| Attack success rate:          | 76.0%  |
| Average perturbed word %:     | 11.43% |
| Average num. words per input: | 133.36 |
| Avg num queries:              | 242.76 |
+-------------------------------+--------+


textattack: Attack time: 114.7847511768341s


## Defining Faster Genetic Algorithm's recipe:

In [ ]:
!pip install torchfile

  Created wheel for torchfile: filename=torchfile-0.1.0-cp37-none-any.whl size=5713 sha256=cd613db44ff3b9bbb207daaaccf6beb887754bdc2113a1c5f4aec7b8813fe016
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built torchfile


In [ ]:
from textattack.constraints.grammaticality.language_models import (
    LearningToWriteLanguageModel,
)
from textattack.constraints.overlap import MaxWordsPerturbed
from textattack.constraints.pre_transformation import (
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import AlzantotGeneticAlgorithm
from textattack.shared.attack import Attack
from textattack.transformations import WordSwapEmbedding
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

transformation = WordSwapEmbedding(max_candidates=8)
        #
        # Don't modify the same word twice or stopwords
        #
constraints = [RepeatModification(), StopwordModification()]
        #
        # Maximum words perturbed percentage of 20%
        #
constraints.append(MaxWordsPerturbed(max_percent=0.2))
        #
        # Maximum word embedding euclidean distance of 0.5.
        #
constraints.append(WordEmbeddingDistance(max_mse_dist=0.5))
        #
        # Language Model
        #
        #
        #
constraints.append(
            LearningToWriteLanguageModel(
                window_size=6, max_log_prob_diff=5.0, compare_against_original=True
            )
        )
        # constraints.append(LearningToWriteLanguageModel(window_size=5))
        #
        # Goal is untargeted classification
        #
goal_function = UntargetedClassification(model_wrapper)
        #
        # Perform word substitution with a genetic algorithm.
        #
search_method = AlzantotGeneticAlgorithm(
            pop_size=60, max_iters=20, post_crossover_check=False
        )

# Now, let's make the attack from the 4 components:
attack1 = Attack(goal_function, constraints, transformation, search_method)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.01 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


## Attacking and displaying results

In [ ]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack1.attack_dataset(dataset, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [02:02<48:55, 122.30s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [06:35<1:15:45, 197.61s/it]

[Succeeded / Failed / Total] 3 / 0 / 3:  12%|█▏        | 3/25 [06:37<48:34, 132.50s/it]

[Succeeded / Failed / Total] 4 / 0 / 4:  16%|█▌        | 4/25 [07:17<38:17, 109.42s/it]

...

[Succeeded / Failed / Total] 16 / 7 / 23:  92%|█████████▏| 23/25 [1:27:11<07:34, 227.44s/it]

[Succeeded / Failed / Total] 16 / 8 / 24:  96%|█████████▌| 24/25 [1:34:06<03:55, 235.28s/it]

[Succeeded / Failed / Total] 17 / 8 / 25: 100%|██████████| 25/25 [1:37:13<00:00, 233.33s/it]

+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 17      |
| Number of failed attacks:     | 8       |
| Number of skipped attacks:    | 0       |
| Original accuracy:            | 100.0%  |
| Accuracy under attack:        | 32.0%   |
| Attack success rate:          | 68.0%   |
| Average perturbed word %:     | 11.6%   |
| Average num. words per input: | 133.36  |
| Avg num queries:              | 4762.84 |
+-------------------------------+---------+


textattack: Attack time: 5833.2535898685455s


# Attacking BERT-base-uncased-yelp-polarity model

In [ ]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
tokenizer = AutoTokenizer("textattack/bert-base-uncased-yelp-polarity")
# Change these

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

dataset = HuggingFaceDataset("yelp_polarity", None, "test") # Change the dataset

## Attacking using our recipe:

In [ ]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack.attack_dataset(dataset, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:02<01:10,  2.93s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [00:05<01:02,  2.70s/it]

[Succeeded / Failed / Total] 3 / 0 / 3:  12%|█▏        | 3/25 [00:05<00:43,  1.98s/it]

[Succeeded / Failed / Total] 4 / 0 / 4:  16%|█▌        | 4/25 [00:07<00:37,  1.78s/it]

...

[Succeeded / Failed / Total] 19 / 4 / 23:  92%|█████████▏| 23/25 [01:33<00:08,  4.07s/it]

[Succeeded / Failed / Total] 20 / 4 / 24:  96%|█████████▌| 24/25 [01:39<00:04,  4.13s/it]

[Succeeded / Failed / Total] 21 / 4 / 25: 100%|██████████| 25/25 [01:40<00:00,  4.01s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 21     |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 16.0%  |
| Attack success rate:          | 84.0%  |
| Average perturbed word %:     | 12.51% |
| Average num. words per input: | 133.36 |
| Avg num queries:              | 238.84 |
+-------------------------------+--------+


textattack: Attack time: 100.20328736305237s


## Attacking using Faster genetic algorithm recipe:

In [ ]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack1.attack_dataset(dataset, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [01:44<41:49, 104.58s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [03:23<39:05, 101.96s/it]

[Succeeded / Failed / Total] 3 / 0 / 3:  12%|█▏        | 3/25 [03:36<26:24, 72.02s/it]

[Succeeded / Failed / Total] 4 / 0 / 4:  16%|█▌        | 4/25 [04:57<26:01, 74.38s/it]

...

[Succeeded / Failed / Total] 16 / 7 / 23:  92%|█████████▏| 23/25 [1:19:40<06:55, 207.86s/it]

[Succeeded / Failed / Total] 17 / 7 / 24:  96%|█████████▌| 24/25 [1:21:20<03:23, 203.36s/it]

[Succeeded / Failed / Total] 18 / 7 / 25: 100%|██████████| 25/25 [1:23:05<00:00, 199.40s/it]

+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 18      |
| Number of failed attacks:     | 7       |
| Number of skipped attacks:    | 0       |
| Original accuracy:            | 100.0%  |
| Accuracy under attack:        | 28.0%   |
| Attack success rate:          | 72.0%   |
| Average perturbed word %:     | 10.12%  |
| Average num. words per input: | 133.36  |
| Avg num queries:              | 5123.12 |
+-------------------------------+---------+


textattack: Attack time: 4985.1249322891235s


# Attacking albert-base-v2-imdb model:

In [ ]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import HuggingFaceDataset

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/albert-base-v2-imdb")
tokenizer = AutoTokenizer("textattack/albert-base-v2-imdb")


model_wrapper = HuggingFaceModelWrapper(model, tokenizer) # Defines our target model

dataset = HuggingFaceDataset("imdb", None, "test") # Change the dataset


Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/4ea52f2e58a08dbc12c2bd52d0d92b30b88c00230b4522801b3636782f625c5b)
textattack: Loading datasets dataset imdb, split test.


In [ ]:
data = []
for i in range(2000):
    data.append((list(dataset.__getitem__(i))[0]['text'],list(dataset.__getitem__(i))[1]))
for i in range(-2000,-1):
    data.append((list(dataset.__getitem__(i))[0]['text'],list(dataset.__getitem__(i))[1]))
data.sort(key = lambda x: len (x[0]))

## Attacking using our recipe:

In [ ]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack.attack_dataset(data, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")





  0%|          | 0/25 [00:00<?, ?it/s]





[Succeeded / Failed / Total] 0 / 0 / 1:   4%|▍         | 1/25 [00:00<00:00, 35.57it/s]





[Succeeded / Failed / Total] 0 / 0 / 1:   8%|▊         | 2/25 [00:00<00:09,  2.37it/s]



[Succeeded / Failed / Total] 0 / 1 / 2:   8%|▊         | 2/25 [00:00<00:09,  2.35it/s]





[Succeeded / Failed / Total] 1 / 1 / 3:  12%|█▏        | 3/25 [00:01<00:11,  1.95it/s]





[Succeeded / Failed / Total] 1 / 1 / 3:  16%|█▌        | 4/25 [00:02<00:13,  1.57it/s]



[Succeeded / Failed / Total] 1 / 2 / 4:  16%|█▌        | 4/25 [00:02<00:13,  1.57it/s]

...





[Succeeded / Failed / Total] 14 / 8 / 23:  96%|█████████▌| 24/25 [00:26<00:01,  1.09s/it]



[Succeeded / Failed / Total] 14 / 9 / 24:  96%|█████████▌| 24/25 [00:26<00:01,  1.09s/it]





[Succeeded / Failed / Total] 15 / 9 / 25: 100%|██████████| 25/25 [00:27<00:00,  1.10s/it]





[Succeeded / Failed / Total] 15 / 9 / 25: : 26it [00:28,  1.11s/it]                      



[Succeeded / Failed / Total] 16 / 9 / 26: : 26it [00:28,  1.11s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 16     |
| Number of failed attacks:     | 9      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 34.62% |
| Attack success rate:          | 64.0%  |
| Average perturbed word %:     | 17.56% |
| Average num. words per input: | 30.42  |
| Avg num queries:              | 63.52  |
+-------------------------------+--------+


textattack: Attack time: 28.861546277999878s


## Attacking using Improved Genetic Algorithm's recipe:

In [ ]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack1.attack_dataset(data, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")





  0%|          | 0/25 [00:00<?, ?it/s]





[Succeeded / Failed / Total] 0 / 0 / 1:   4%|▍         | 1/25 [00:00<00:00, 33.46it/s]





[Succeeded / Failed / Total] 0 / 0 / 1:   8%|▊         | 2/25 [01:38<18:57, 49.47s/it]



[Succeeded / Failed / Total] 0 / 1 / 2:   8%|▊         | 2/25 [01:38<18:57, 49.48s/it]





[Succeeded / Failed / Total] 0 / 1 / 2:  12%|█▏        | 3/25 [02:09<15:48, 43.10s/it]



[Succeeded / Failed / Total] 1 / 1 / 3:  12%|█▏        | 3/25 [02:09<15:48, 43.10s/it]





[Succeeded / Failed / Total] 1 / 1 / 3:  16%|█▌        | 4/25 [05:10<27:12, 77.73s/it]



[Succeeded / Failed / Total] 1 / 2 / 4:  16%|█▌        | 4/25 [05:10<27:12, 77.73s/it]

...





[Succeeded / Failed / Total] 9 / 13 / 23:  96%|█████████▌| 24/25 [58:45<02:26, 146.88s/it]



[Succeeded / Failed / Total] 9 / 14 / 24:  96%|█████████▌| 24/25 [58:45<02:26, 146.88s/it]





[Succeeded / Failed / Total] 9 / 14 / 24: 100%|██████████| 25/25 [59:26<00:00, 142.64s/it]



[Succeeded / Failed / Total] 10 / 14 / 25: 100%|██████████| 25/25 [59:26<00:00, 142.64s/it]





[Succeeded / Failed / Total] 10 / 14 / 25: : 26it [1:05:00, 150.02s/it]                    



[Succeeded / Failed / Total] 10 / 15 / 26: : 26it [1:05:00, 150.02s/it]

+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 10      |
| Number of failed attacks:     | 15      |
| Number of skipped attacks:    | 1       |
| Original accuracy:            | 96.15%  |
| Accuracy under attack:        | 57.69%  |
| Attack success rate:          | 40.0%   |
| Average perturbed word %:     | 18.3%   |
| Average num. words per input: | 30.42   |
| Avg num queries:              | 3358.08 |
+-------------------------------+---------+


textattack: Attack time: 3900.52823805809s
